In [85]:
""" !pip install opendartreader
!pip3 install --upgrade opendartreader
!pip3 install beautifulsoup4 """

' !pip install opendartreader\n!pip3 install --upgrade opendartreader\n!pip3 install beautifulsoup4 '

## 자본, 자산, 당기순이익 조회

#### 코드 참고자료 링크 : https://psystat.tistory.com/117?category=1176931 

#### 당기순이익이 아닌, 영업이익을 조회해야 하는 이유
#### 링크 : https://m.joseilbo.com/news/view.htm?newsid=382751

|변수명|변수설명|비고|
|:------:|:------:|:----------|
|corp|종목코드||
|bsns_year|사업연도||
|reprt_code|보고서 코드|11011 : 사업보고서|
|상동|상동|11012 : 2분기보고서|
|상동|상동|11013 : 1분기보고서|
|상동|상동|11014 : 3분기보고서|

In [1]:
from datetime import datetime
import OpenDartReader
import pandas as pd
import numpy as np
import csv

In [2]:
api_key = '4b66b1aee7205d6084aae7f8ceaa7c8231c3d38a'
dart = OpenDartReader(api_key)

In [3]:
fs_2019 = dart.finstate_all(corp= '000660', bsns_year = '2020', fs_div='CFS', reprt_code = 11011)
fs_2020_3Q = dart.finstate_all(corp = '000660', bsns_year = '2020', fs_div='CFS', reprt_code=11014)

In [4]:
fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['BS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_Equity']), ]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
51,20201116001609,11014,2020,00164779,BS,재무상태표,ifrs-full_Equity,자본총계,-,제 73 기 3분기말,50499059000000,제 72 기말,47943195000000,54,NaN,NaN,NaN,NaN


In [15]:
fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['BS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_Liabilities']), ]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
43,20201116001609,11014,2020,00164779,BS,재무상태표,ifrs-full_Liabilities,부채총계,-,제 73 기 3분기말,18768114000000,제 72 기말,16846299000000,45,NaN,NaN,NaN,NaN


In [16]:
fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['CIS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), ]  #isin(['CIS']) 입니다. 링크상에서는 'IS' 로 인자값을 주지만, 이러면 나오질 않습니다. 조심

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
72,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,ifrs-full_ProfitLossAttributableToOwnersOfParent,지배기업의 소유주지분,-,제 73 기 3분기,1077262000000,NaN,NaN,22,2988306000000,제 72 기 3분기,493201000000,2131483000000


In [7]:
equity = int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['BS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",","")) # 당기자본(자본총계)
liability = int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['BS'])& fs_2020_3Q['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",",""))
assets = equity + liability
assets

69267173000000

In [20]:
profit_2019_3Q = int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['CIS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",",""))
profit_2019 = int(fs_2019.loc[fs_2019['sj_div'].isin(['CIS']) & fs_2019['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", ""))
profit_2020_3Q = int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['CIS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",",""))
profit = (profit_2019 - profit_2019_3Q) + profit_2020_3Q
profit

5611925000000

## 주주관련 조회

|변수명|변수설명|
|:------:|:------:|
|shrholdr_co|주주수|
|shrholdr_tot_co|전체 주주수|
|shrholdr_rate|주주 비율|
|hold_stock_co|보유 주식수|
|stock_tot_co|총발행 주식수|
|hold_stock_rate|보유 주식 비율|

#### year 값 가능 범위 (2019, 2018,2017,2016,2015) 이외에는 조회값이 나오질 않네요.. 

In [24]:
year = 2019

In [25]:
total_num_shares = dart.report('000660', '소액주주', year)
total_num_shares

,rcept_no,corp_cls,corp_code,corp_name,se,shrholdr_co,shrholdr_tot_co,shrholdr_rate,hold_stock_co,stock_tot_co,hold_stock_rate
0,20200330004441,Y,00164779,SK하이닉스,소액주주,"181,783",-,99.99%,"441,033,663",-,60.58%


## 지표계산(EPS, BPS, ROE)

In [26]:
df = pd.read_csv('sk하이닉스_2759_20210413.csv', encoding = 'cp949')
df.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2021/04/12,137500,-2500,-1.79,140500,140500,136000,3878457,534353588500,100100325187500,728002365
1,2021/04/09,140000,-4000,-2.78,144000,145000,139500,3276077,464472291000,101920331100000,728002365
2,2021/04/08,144000,500,0.35,143500,146000,142500,3774932,543862222000,104832340560000,728002365
3,2021/04/07,143500,500,0.35,146000,146000,142500,2424687,348944024000,104468339377500,728002365
4,2021/04/06,143000,0,0.00,147000,147000,141500,3753217,540574718000,104104338195000,728002365


In [27]:
df['상장주식수'].iat[0]

728002365

In [28]:
total_stocks = int(df['시가총액'].iat[0]/df['종가'].iat[0]) #상장주식수 = 시가총액/종가

In [32]:
EPS = profit/total_stocks
PER = df['종가'].iat[0]/EPS
print('EPS: {}, PER: {}'.format(int(round(EPS)), round(PER, 2)))

EPS: 7709, PER: 17.84


In [33]:
BPS = equity/total_stocks
PBR = df['종가'].iat[0]/BPS
print('BPS: {}, PBR: {}'.format(int(round(BPS)), round(PBR, 2)))

BPS: 69367, PBR: 1.98


In [34]:
ROE = PBR/PER
ROA = profit/assets
print('ROE: {}, ROA: {}'.format(int(round(ROE*100, 2)), round(ROA*100, 2)))

ROE: 11, ROA: 8.1


## 매출 / 영업이익 조회

In [1]:
df_sales = dart.finstate_all('000660', 2019)
df_sales.loc[df['sj_div'].isin(['CIS']) & df_sales['account_nm'].isin(['매출액']), ]

NameError: name 'dart' is not defined

In [ ]:
df_sales.loc[df['sj_div'].isin(['CIS']) & df_sales['account_nm'].isin(['영업이익']), ]